# 1.Installing required dependencies

In [1]:
! pip install matplotlib==3.1.1
! pip install igraph
! pip install colorama
# ! pip install 'fsspec>=0.3.3'
! pip install d2l==0.17.5
! pip install --upgrade mxnet
# ! pip install --upgrade mxnet-cu100

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.1-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.1
    Uninstalling matplotlib-3.5.1:
      Successfully uninstalled matplotlib-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
d2l 0.17.5 requires matplotlib==3.5.1, but you have matplotlib 3.1.1 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.1
    Uninstalling matplotlib-3.1.1:
      Successfully uninstalled matplotlib-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


✅ **Restart your kernel before running rest of the code**
 

---



---



---




# 2. Importing required libraries

In [2]:
import os
import networkx as nx
import networkx.algorithms.community as nx_comm
import pandas as pd
import numpy as np
from colorama import Fore
import networkx as nx
# from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import time
from d2l import mxnet as d2l
from sklearn.model_selection import train_test_split

# 3. Loading and preprocessing the dataset

In [3]:
#  در این قسمت از کد با استفاده از کتابخانه از پیش  آمده شده دیتا ست موی لنز خوانده شده است 
# movie lense 100k
d2l.DATA_HUB['ml-100k'] = (
    'https://files.grouplens.org/datasets/movielens/ml-100k.zip',
    'cd4dcac4241c8a4ad7badc7ca635da8a69dddb83')

def read_data_ml100k():
    data_dir = d2l.download_extract('ml-100k')
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names,
                       engine='python')
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items

In [4]:
# با استفاده از فانکشن نوشته شده در بلوک کد بالا  دیتافریم متناظر با موی لنز در این قسمت خوانده میشود. 
data, num_users, num_items = read_data_ml100k()
data2 = data.copy(deep=True)
sparsity = 1 - len(data) / (num_users * num_items)
print(Fore.CYAN,f'number of users: {num_users}, number of items: {num_items}')
print(f'matrix sparsity: {sparsity:f}')
print('\n')
#  پنج سطر ابتدایی دیتا ست در این جاپرینت میشود
print(Fore.BLACK,data.head(5))
print('\n')
print(Fore.BLUE,'data shape is: ',data.shape)
print('\n')


 number of users: 943, number of items: 1682
matrix sparsity: 0.936953


    user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


 data shape is:  (100000, 4)




In [5]:
#  در این قسمت از کد اطلاعات مربوط به ژانر فیلمها از فایل مستقلی خوانده میشود و ارتباط فیلمها بر حسب ژانر مورد بررسی قار میگیرد.
data_dir = d2l.download_extract('ml-100k')
#  در خط زیر اسم ستونهای مربوط به دیتاست مشخص میشود.
#  هر کدام از ستونهای مشخص شده با حرف جی انگلیسی بیانگر یک ژانر مشخص میباشد.
#  متناسب با محتوای هر فیلم به چند ژانر مرتبط با آن عدد یک نسبت داده میشود. 
m_cols =['id','movie','t','n','site','j0','j1','j2','j3','j4','j5','j6','j7','j8','j9','j10','j11','j12','j13','j14','j15','j16','j17','j18']
# اطلاعات مربوط به ژانر فیلم های موجود در دیتاست در خط بعد خوانده میشود
generes = pd.read_csv(os.path.join(data_dir, 'u.item'),sep='|', names=m_cols , encoding='latin-1')
def clean(x):
    x=x.split('(')[0]
    return(x)
generes['movie'] =generes['movie'].apply(clean)
generes= generes.set_index('id')
#  ستونهایی از دیتا ست که در این قسمت مورد نیاز نمیباشند حذف گردیده اند.
generes.drop(columns = ['movie','t','n','site','j0'],inplace =True)
generes
#  . کورولیشن بین هر دو فیلم ممکن در این قسمت محاسبه میشود
#  خروجی نهایی این قسمت یک دیتا فریم میباشد که اعداد موجود در آن ارتباط بین فیلمها را مشخص میسازد. 
#  مطابق انتظار درایه متناظر با قطر اصلی این دیتافریم برابر با یک به دست آمده است
movies_by_geners_corr = generes.transpose().corr()
movies_by_geners_corr 


id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.200000,-0.108465,0.200000,-0.200000,-0.108465,-0.158114,0.600000,-0.108465,-0.158114,...,-0.158114,-0.108465,-0.108465,-0.108465,-0.108465,-0.108465,-0.158114,-0.158114,0.542326,-0.108465
2,-0.200000,1.000000,0.542326,0.200000,0.200000,-0.108465,-0.158114,-0.200000,-0.108465,-0.158114,...,0.790569,-0.108465,-0.108465,-0.108465,-0.108465,-0.108465,0.316228,-0.158114,-0.108465,-0.108465
3,-0.108465,0.542326,1.000000,-0.108465,0.542326,-0.058824,-0.085749,-0.108465,-0.058824,-0.085749,...,0.685994,-0.058824,-0.058824,-0.058824,-0.058824,-0.058824,0.685994,-0.085749,-0.058824,-0.058824
4,0.200000,0.200000,-0.108465,1.000000,0.200000,0.542326,0.316228,0.600000,0.542326,0.316228,...,0.316228,0.542326,0.542326,0.542326,0.542326,0.542326,-0.158114,0.316228,0.542326,0.542326
5,-0.200000,0.200000,0.542326,0.200000,1.000000,0.542326,0.316228,0.200000,0.542326,0.316228,...,0.316228,0.542326,0.542326,0.542326,0.542326,0.542326,0.316228,0.316228,-0.108465,0.542326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,-0.108465,-0.108465,-0.058824,0.542326,0.542326,1.000000,0.685994,0.542326,1.000000,0.685994,...,-0.085749,1.000000,1.000000,1.000000,1.000000,1.000000,-0.085749,0.685994,-0.058824,1.000000
1679,-0.158114,0.316228,0.685994,-0.158114,0.316228,-0.085749,-0.125000,-0.158114,-0.085749,-0.125000,...,0.437500,-0.085749,-0.085749,-0.085749,-0.085749,-0.085749,1.000000,0.437500,-0.085749,-0.085749
1680,-0.158114,-0.158114,-0.085749,0.316228,0.316228,0.685994,0.437500,0.316228,0.685994,0.437500,...,-0.125000,0.685994,0.685994,0.685994,0.685994,0.685994,0.437500,1.000000,-0.085749,0.685994


In [6]:
#  در این قسمت مجددا به سراغ دیتاست اصلی که در آن اطلاعات مربوط به نمرات کاربران به فیلمها اورده شده است میرویم
start = time.time()
#  در خط زیر عملیات 
# one hot encoding 
#  انجام میپذیرد. لیست یونیکی از تمام فیلمهای موجود در کل فایل با اجرای کد زیر برای ما حاصل میشود
df_i = pd.get_dummies(data['item_id'])
# با استفاده از لیست اعداد ذخیره شده در گام قبل به ازای هر فیلم یک ستون به دیتا اضافه میکنیم 
data[ list(df_i.columns) ] = df_i
data.drop('item_id', inplace=True, axis=1)
end = time.time()
print(f"Runtime of the program is {end - start}")
data.head()    


Runtime of the program is 3.218806266784668


,user_id,rating,timestamp,1,2,3,4,5,6,7,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,196,3,881250949,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,186,3,891717742,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22,1,878887116,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,244,2,880606923,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,166,1,886397596,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#  ستوهای دیتای به دست آمده در کد بالا هر یک مشخص کننده یک فیلم میباشند
# ایندکس مربوط به سطرهای دیتای بالا هر کدام بیانگر کاربری هستند که به فیلم مد نظر ما نمره داده اند
# هر فیلمی که توسط کاربر به ان نمره داده شده است با عدد یک مشخص  شده است
#  در ادامه کد زیر میخواهیم به دیتایی برسیم که تنها حاوی اطلاعات مربوط به شماره کاربران و شماره فیلمها باشد 
#  هر آرایه از دیتای نهایی نمره کاربر به فیلم متناظر را مشخص میکند
for i in range(1,data.shape[1]-3):
    
    data[i] = data[i]*data['rating']

    if i %500 ==0  :
      end = time.time()
      print(f"Runtime of the program is {end - start}", ' for i =',i,'/',data.shape[1])

data.drop('timestamp', inplace=True, axis=1)
data.drop('rating', inplace=True, axis=1)

data

Runtime of the program is 25.335283517837524  for i = 500 / 1685
Runtime of the program is 39.64918065071106  for i = 1000 / 1685
Runtime of the program is 45.580721378326416  for i = 1500 / 1685


,user_id,1,2,3,4,5,6,7,8,9,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,196,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,186,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,244,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,166,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,716,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,276,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#  این قسمت از کد به منظور ارزیابی توزیع دیتا میباشد.
# دیتا سریز تولید شده در این قسمت که نام آن در خط زیر امده است 
# A
#  مشخص میکند که به هر فیلم چه تعداد از کاربران نمره داده اند. 
data_ = data.groupby('user_id').sum()
A = data_[data_ !=0].count(axis=0)
print(A)
B  = A[A<3]
C=A[A==1]
print(Fore.RED,'movies with less than 3 rates: ',B.shape[0])
print(Fore.BLUE,'movies with just 1 rate : ',C.shape[0])
# data_[data_ !=0].count().hist()

1       452
2       131
3        90
4       209
5        86
       ... 
1678      1
1679      1
1680      1
1681      1
1682      1
Length: 1682, dtype: int64
 movies with less than 3 rates:  209
 movies with just 1 rate :  141


In [9]:
# در این قسمت مشابه با بلوک کد قبل به ارزیابی دیتا میپردازیم و مشخص میکنیم که هر از کاربران موجود در دیتا ست 
#  به چند فیلم رای داده اند
A = data_[data_ !=0].count(axis=1)
print(A)
B  = A[A<10]
C=A[A==1]
print(Fore.RED,'users with less than 3 rates: ',B.shape[0])
print(Fore.BLUE,'users with just 1 rate : ',C.shape[0])
# data_[data_ !=0].count(axis=1).hist()

user_id
1      272
2       62
3       54
4       24
5      175
      ... 
939     49
940    107
941     22
942     79
943    168
Length: 943, dtype: int64
 users with less than 3 rates:  0
 users with just 1 rate :  0


# 4. test train split

In [10]:
#  در این قسمت به سراغ کتابخانه سایکت لرن میرویم و دیتای موجود را به صورت رندوم به دو دسته دیتای یادگیری 
#  و دیتای ارزیابی تقسیم میکنیم 
# نود درصد دیتا به منظور آموزش و ده درصد آن به منظور ارزیابی مدل به کار گرفته خواهند شد

train, test = train_test_split( data, test_size=0.10, random_state=42,shuffle=True)
#  با استفاده از دستور گروپ بای نظرات هر کاربر را به یک خط از دیتا ست منتقل میکنیم 
#  قبل از اجرای خط زیر هر رای داده شده توسط هر کاربری یک سطر از دیتاست را تشکیل میداد 
train = train.groupby('user_id').sum()

train

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5,3,4,3,3,0,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
940,0,0,0,2,0,0,4,0,3,0,...,0,0,0,0,0,0,0,0,0,0
941,5,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 5. initial community

In [11]:
# گروهبندی اولیه بر اساس پیرسون کورولیشن بین فیلمها محاسبه میشود
# نکته مهمی که میایست به آن توجه کرد حذف صفرها با مقدار نامعین پیش از انجام محاسبات کورولیشن میباشد\
#  دلیل این موضوع در فایل صوتی تشریح میشود
tem =  train.replace(0, np.NaN)
corr_between_movies = tem.corr()
corr_between_movies.fillna(0,inplace=True)
#  as NaN corr indicatates none of  users voted both films we repalce it with zero
# هر دو فیلمی که نتیجه کورولیشن بین انها نامعین باشد در واقع مشخص میسازد که کاربر مشترکی به آن دو  فیلم رای نداده است 
#  و نمیتوان قضاوت دقیقی از ارتباط میان ان دو داشت 
#  کورولیشن نظیر در این فیلمها برابر با صفر در نظر گرفته میشود 
corr_between_movies

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.000000,0.119688,0.193263,0.080033,0.367078,0.641555,0.235754,0.265850,0.064203,0.105623,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.119688,1.000000,0.107736,0.229167,0.054939,-0.500000,0.243767,0.331389,-0.162939,0.083756,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.193263,0.107736,1.000000,-0.137896,0.153449,0.884652,0.088885,-0.055548,0.019973,0.100372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.080033,0.229167,-0.137896,1.000000,-0.153493,0.558744,0.175420,0.239603,0.329635,0.047525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.367078,0.054939,0.153449,-0.153493,1.000000,0.000000,0.098315,0.208098,0.038971,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1681,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#  در اینجا بر خلاف مقاله گروهبندی در چند سطح صورت پذیرفته است که این موضوع در فایل صوتی تشریح میگردد
start = time.time()
num =train.shape[1]
total_indices = [i for i in range(1,num)]
tot = 0

#  netorkx
#  نام فوق کتابخانه استفاده شده به منظور تحلیل گراف در پایتون میباشد که در این کد از آن بهره خواهیم گرفت
#  first level
# دستور زیر یک گراف اولیه خالی و بدون عضو را مشخص میکند
G1  = nx.Graph()
#  مقادیر زیر در فایل صوتی تشریح شده اند
min_num_same_rate1 = 4
min_sim1 = 0.9


# second level
G2  = nx.Graph()
min_num_same_rate2 = 4
min_sim2 = 0.60

# third level 
G3  = nx.Graph()
min_num_same_rate3 = 2
min_sim3 = 0.01


#  در این قسمت میخوایم هر فیلم را نسبت به سایر فیلمها بسنجیم بدین منظور با استفاده از
#  یک حلقه فور بر روی فیلمهای مختلف لوپ میزنیم 
#  نکته کلیدی در این قسمت این است که کد میبایست تا حد ممکن به صورت وکتورایز نوشته شود و در عملیاتهای داخل
#  حلقه فور کلی تا حد ممکن از سایر حلقه ها استفاده نکنیم 
# به کارگیری حلقه های تو در تو در این قسمت منجر به کندی زیاد کد خواهد شد که از ان اجتناب شده است 
for i in range (1,num):
      is_conected =0
      
      both_zero = train.mul(train[i], axis = 0)
      same_rate = train.sub(train[i],axis=0).abs()
      same_rate[both_zero  ==0] =100
     
      same_rate_count = same_rate[same_rate<3].count()
       
      
      indices1 = list(set(list(same_rate_count[same_rate_count>= min_num_same_rate1].index))-set([i]))
      indices2 = list(set(list(same_rate_count[same_rate_count.between(min_num_same_rate2,min_num_same_rate1 ,inclusive='left')].index))-set([i]))
      indices3 = list(set(list(same_rate_count[same_rate_count.between(min_num_same_rate3,min_num_same_rate2 ,inclusive='left') ].index))  -set([i]))
      
      
      
      
      
      new_indices1 = [x-1 for x in indices1]
      corr_i_j = corr_between_movies.iloc[i-1,new_indices1] 
      tem =  corr_i_j[corr_i_j>=min_sim1]
      similar_movies = list(tem.index)
      similar_movies_corr = list(tem)
      if len(similar_movies)!= len(similar_movies_corr):
        print('sth is wrong 2 !! ')
      related_item=[('m' +str(i),'m' +str(similar_movies[mm]),similar_movies_corr[mm]) for mm in range(len(similar_movies))]
      
      if related_item:
              is_conected =1
              # print(i,related_item)
              G1.add_weighted_edges_from(related_item)



      new_indices2 = [x-1 for x in indices2]
      corr_i_j_2 = corr_between_movies.iloc[i-1,new_indices2] 
      tem2 =  corr_i_j_2[corr_i_j_2>=min_sim2]
      similar_movies2 = list(tem2.index)
      similar_movies_corr2 = list(tem2)
      if len(similar_movies2)!= len(similar_movies_corr2):
        print('sth is wrong 2!! ')
      related_item2=[('m' +str(i),'m' +str(similar_movies2[mm]),similar_movies_corr2[mm]) for mm in range(len(similar_movies2))]
      if related_item2:
              is_conected =1
              G2.add_weighted_edges_from(related_item2)

      new_indices3 = [x-1 for x in indices3]
      corr_i_j_3 = corr_between_movies.iloc[i-1,new_indices3] 
      tem3=  corr_i_j_3[corr_i_j_3>=min_sim3]
      similar_movies3 = list(tem3.index)
      similar_movies_corr3 = list(tem3)
      if len(similar_movies3)!= len(similar_movies_corr3):
        print('sth is wrong 3!! ')
      related_item3=[('m' +str(i),'m' +str(similar_movies3[mm]),similar_movies_corr3[mm]) for mm in range(len(similar_movies3))]
      if related_item3:
              is_conected =1
              G3.add_weighted_edges_from(related_item3)


      


      if i %200 ==0  :
        end = time.time()
        print(Fore.CYAN+f"time is: {end - start}",' for i =',i,'/',data.shape[1])




  

time is: 24.370933294296265  for i = 200 / 1683
time is: 48.75023436546326  for i = 400 / 1683
time is: 72.76997685432434  for i = 600 / 1683
time is: 97.08626008033752  for i = 800 / 1683
time is: 120.8070080280304  for i = 1000 / 1683
time is: 145.45914387702942  for i = 1200 / 1683
time is: 168.91412043571472  for i = 1400 / 1683
time is: 192.39182901382446  for i = 1600 / 1683


In [13]:
# from networkx.algorithms.community.centrality import girvan_newman
# communities = girvan_newman(G1)
# node_groups = []
# for com in next(communities):
#   node_groups.append(list(com))

# print(node_groups)

# from networkx.algorithms.community import label_propagation_communities
# groups = label_propagation_communities(G)

# from networkx import edge_betweenness_centrality as betweenness
# from networkx.algorithms.community import girvan_newman
# def most_central_edge(G):
#     centrality = betweenness(G, weight="weight")
#     return max(centrality, key=centrality.get)

# from networkx.algorithms.community import asyn_lpa_communities
# groups = asyn_lpa_communities(G,weight='weight')

# from networkx.algorithms.community import greedy_modularity_communities
# groups = greedy_modularity_communities(G,weight='weight')

# from networkx.algorithms.community import k_clique_communities
# c = list(k_clique_communities(G4, ))

# from networkx.algorithms.community import kernighan_lin_bisection
# groups = kernighan_lin_bisection(G3, partition=None, max_iter=10, weight='weight', seed=None)


In [14]:

from networkx.algorithms.community import asyn_fluidc
gr ={}
gr[1] = asyn_fluidc(G1, 20, max_iter=100, seed=15)
gr[2] = asyn_fluidc(G2, 15, max_iter=100, seed=15)
gr[3]= asyn_fluidc(G3, 10, max_iter=100, seed=20)
# gr[4] = asyn_fluidc(G4, 30, max_iter=100, seed=30)

# #  do not run this section twice
A =0
groups_dict= {}
# sparse_group ={}
i=0
v =0
for j  in range(1,4):
  #  if j ==4:
  #    for h in gr[j]:
  #      sparse_group[v] = h 
  #      v+=1
    

  #  else:
    for h in gr[j]:
  
      groups_dict[i] =h
      i+=1

assert len(groups_dict) != 0, "sth is wrong, you run the code twice"
# assert len(sparse_group) != 0, "sth is wrong, you run the code twice"

#6.users community


In [15]:
users_train = train.transpose()
tem = users_train.replace(0, np.NaN)
users_corr = users_train.corr()
start = time.time()

min_user_sim =0.6
num =users_train.shape[1]
G_users = nx.Graph()

for i in range (num): 
      
     
      for j in range (i+1,num):
        
        if users_corr.iloc[i,j]>min_user_sim:
            user1= i+1
            user2 = j+1
            G_users.add_edge(user1,user2) 
     
    
                        

      if i %200 ==0  :
        end = time.time()
        print(Fore.BLACK+f"time is: {end - start}", ' for i =',i)


time is: 0.024341583251953125  for i = 0
time is: 4.490167140960693  for i = 200
time is: 7.88861608505249  for i = 400
time is: 10.206949472427368  for i = 600
time is: 11.505855083465576  for i = 800


In [16]:
start = time.time()
all_users =train.shape[0]
all_users =[i for i in range(1,train.shape[0]+1)]
user_path = np.zeros((len(all_users),len(all_users)))
for i in range(len(all_users)):
  for j in range(len(all_users)):
    try:
          user_path[i-1,j-1]  = (nx.shortest_path_length( G_users,source = all_users[i], target=all_users[j]))
    except:
      user_path[i-1,j-1] =10
 
  
  if i %200 ==0  :
        end = time.time()
        print(Fore.RED+f"time is: {end - start}", ' for i =',i,'/',len(all_users))

time is: 0.005542278289794922  for i = 0 / 943
time is: 0.3911745548248291  for i = 200 / 943
time is: 0.7722747325897217  for i = 400 / 943
time is: 1.1663322448730469  for i = 600 / 943
time is: 1.5516016483306885  for i = 800 / 943


# 7. Test

In [17]:
def prediction_func(traindataset,neighbor_movies,similar_users,target_item,target_id):
          filtered = traindataset [neighbor_movies]
         
          filtered2 =  filtered[ filtered.index.isin(similar_users)]
          filtered2 =  filtered2.replace(0, np.NaN)
          filtered_corr = filtered2.corr()
          
          s = filtered_corr[target_item]
          # print('filtered_corr')
          # print(filtered_corr)
          s[s<0] = 0
          s = s.fillna(0)
          # print('\n\n s')
          # print(s)
          r = filtered[filtered.index == target_id]

          #  اونجاهایی که ار برابر صفره نباید اس ها جمع بشن 
          y = s.transpose().mul(r, axis = 0)
          y =y.to_numpy().ravel()
          s= s.to_numpy().ravel()
          dem= s[y!=0].sum()
          # print(dem)
          if dem != 0:
            pr =((r.dot(s).iloc[0])/dem)
       
          else :
            pr = False

          return pr 
         


In [18]:
#############
train_n = train.replace(0, np.NaN)
users_avg = train_n.mean(axis=1)
users_avg_total =users_avg.mean()
users_avg_total
users_avg[2] /users_avg_total

1.0422567488978818

In [19]:
target_rates = np.zeros([test.shape[0]])
start = time.time()
predictions =np.zeros([test.shape[0]])
counter =0
m_list =[]
without_gr =[]
test_data =  data2.loc[test.index]
train_n = train.replace(0, np.NaN)

for _, row in test_data.iterrows():
      
      
    
      target_id = (row['user_id'])
      # neighbor users
      index = np.where(user_path[target_id-1]==1)
      similar_users = index +np.array([1])
      similar_users = list(similar_users[0])
      target_item =(row['item_id'])      

      tem = 'm'+str(target_item)
      target_group =[]
      for i in range (len(groups_dict)):
        if tem in groups_dict[i]:
          
          target_group.append(i) 
          
      
      if target_group :
        

        for ie in range(len(target_group)):
          a=0      
          if a==0:
            neighbor_movies = list(groups_dict[target_group[ie]])
            neighbor_movies = [int(neighbor_movies[i].split('m')[1]) for i in range(len(neighbor_movies))]
            pr1 =  prediction_func(train,neighbor_movies,similar_users,target_item,target_id)
            if pr1:
              pr1 =(pr1 *users_avg[target_id] /users_avg_total)
              a=1
              if pr1 >5:
                  pr1 =5
              break
             
            else:
              index2 = np.where(user_path[target_id-1]==2)
              similar_users2 = index2 +np.array([1])
              similar_users2 =list(set(list(similar_users2[0])+similar_users)) 
              pr1 =  prediction_func(train,neighbor_movies,similar_users2,target_item,target_id)
              
              if pr1 :
                print(Fore.RED,ie)
                a=1
                pr1 =(pr1 *users_avg[target_id] /users_avg_total)
                if pr1 >5:
                  pr1 =5
                break
               


          else :
              break

     
      for ie2 in([0.95,0.7,0.45,0,-0.8]): 
          a= 0
          if a==0: 
            neighbor_genres =list(movies_by_geners_corr[target_item][movies_by_geners_corr[target_item]>ie2].index )
            
            # neighbor_genres =list(set(set(neighbor_genres)-{target_item}))
            pr_g =  prediction_func(train,neighbor_genres,similar_users,target_item,target_id)
            if  pr_g:
              a=1
              break
            elif ie2<0.1 :
              
              similar_users3 = all_users
              pr_g = prediction_func(train,neighbor_genres,similar_users3,target_item,target_id)
              if  pr_g:
                a=1
                break
          else:
            break

      # assert pr_g or pr1 ,f"prediction vlaue is not possible"


      if pr_g and  pr1 :
        
            pr =0.6*pr1 +0.4*pr_g
      elif pr_g and pr1 ==False:
          pr =pr_g
      elif pr_g ==False and pr1 :
          pr = pr1
      elif pr_g==False and pr1 ==False:
          
          m_list.append(target_item)
          pr =np.NaN


      predictions[counter]= pr
      target_rates[counter] = (row['rating'])
      if np.abs(predictions[counter]-target_rates[counter])>2:
                print(Fore.RED+'counter:' ,counter, 'user_id:',row['user_id'],"pre: ",
                      predictions[counter],"tg_ra: ",target_rates[counter],Fore.GREEN+'target_group:',
                      target_group)
      elif  np.abs(predictions[counter]-target_rates[counter])>=1:
                print(Fore.BLUE+'counter:' ,counter, 'user_id:',row['user_id'],"pre: ", predictions[counter],"tg_ra: ",target_rates[counter],Fore.GREEN+'target_group:',target_group)  
      else :
            print(Fore.BLACK+'counter:' ,counter, 'user_id:',row['user_id'],"pre: ", predictions[counter],"tg_ra: ",target_rates[counter],Fore.GREEN+'target_group:',target_group)
      counter+=1
      if counter %200 ==0  :
        end = time.time()
        print(Fore.CYAN+f"time is: {end - start}", ' for i =',counter,'/',(test_data.shape[0]))


      if target_group ==-1:
        print(Fore.RED+'\n\n ______________this{} item doen not exist in opur dataset____________'.format(target_item))




Streaming output truncated to the last 5000 lines.
counter: 5025 user_id: 486 pre:  3.2633310851336286 tg_ra:  3.0 target_group: [9, 26, 41]
counter: 5026 user_id: 15 pre:  2.912305710512847 tg_ra:  1.0 target_group: [17, 34, 41]
counter: 5027 user_id: 363 pre:  3.017782205832296 tg_ra:  1.0 target_group: [19, 25, 43]
counter: 5028 user_id: 450 pre:  3.8054474012498334 tg_ra:  4.0 target_group: [4, 27, 38]
counter: 5029 user_id: 110 pre:  2.9415221254953754 tg_ra:  1.0 target_group: [5, 20, 38]
counter: 5030 user_id: 579 pre:  3.5955313842308936 tg_ra:  4.0 target_group: [15, 31, 36]
counter: 5031 user_id: 373 pre:  3.5679332858288175 tg_ra:  3.0 target_group: [10, 30, 40]
counter: 5032 user_id: 589 pre:  3.9099153983708455 tg_ra:  5.0 target_group: [6, 22, 37]
counter: 5033 user_id: 286 pre:  3.9266030389275075 tg_ra:  4.0 target_group: [0, 25, 37]
counter: 5034 user_id: 493 pre:  3.847247921890036 tg_ra:  3.0 target_group: [1, 27, 44]
counter: 5035 user_id: 276 pre:  3.75667886486892

In [20]:
target_rates[np.isnan(predictions)] =0
predictions[np.isnan(predictions)] = 0
mse = (np.square(target_rates - predictions)).mean()
print('mse in our test set is: ',mse)

mse in our test set is:  1.0264932594238159


In [21]:
print('prediction mean is :',predictions.mean())
print('rates mean is :',target_rates.mean())

prediction mean is : 3.5173632878089527
rates mean is : 3.5065
